Connecting data

In [ ]:
%pip install pandas
%pip install folium
%pip install mapclassify
%pip install geopandas
%pip install selenium


In [48]:
import pandas as pd
import geopandas as gpd
import folium
from IPython.display import display
import matplotlib.pyplot as plt
import os

In [3]:
# Load GeoJSON data
geojson_data = gpd.read_file("data/administrative_units/gradovi_opcine_zupanije.geojson")

In [4]:
# Ensure that the key columns are named consistently
geojson_data['Županija'] = geojson_data['text_right'].str.strip()
geojson_data['Grad/općina'] = geojson_data['text_left'].str.strip()

In [49]:
def create_interactive_population_map(df, color_column_name, folder_name):
    # Merge dataframes based on the matching columns
    df = geojson_data.merge(df, on=['Županija', 'Grad/općina'], how='inner')

    # Ensure the necessary columns are present
    if 'Županija' not in df.columns or 'Grad/općina' not in df.columns or 'Ukupno' not in df.columns:
        raise ValueError("The CSV must contain 'Županija', 'Grad/općina', and 'Ukupno' columns.")

    # Convert Timestamp columns to strings
    for col in df.select_dtypes(include=['datetime64[ns, UTC]', 'datetime64[ns]']).columns:
        df[col] = df[col].astype(str)

    # Prepare hover information with sorted percentage columns
    def create_hover_info(row):
        percentage_columns = [col for col in df.columns if col.endswith('%')]
        sorted_percentages = sorted(
            [(col, row[col]) for col in percentage_columns if not pd.isna(row[col])],
            key=lambda x: x[1], reverse=True
        )
        
        hover_text = (
            f"Županija: {row['Županija']}<br>" 
            f"Grad/Općina: {row['Grad/općina']}<br>" 
            f"Ukupno: {row['Ukupno']}<br>"
        )
        hover_text += ''.join([f"<br>{col}: {value}" for col, value in sorted_percentages])
        return hover_text

    df['hover_info'] = df.apply(create_hover_info, axis=1)

    # Check if the color_column_name exists in the DataFrame
    if color_column_name not in df.columns:
        raise ValueError(f"The specified color column '{color_column_name}' does not exist in the CSV file.")

    # Create a color map based on the specified color column
    min_value = df[color_column_name].min()
    max_value = df[color_column_name].max()

    colormap = folium.LinearColormap(
        colors=['blue', 'yellow', 'red'],
        vmin=min_value, 
        vmax=max_value,
        caption=color_column_name
    )

    # Add color column based on the value of the specified column
    df['color'] = df[color_column_name].apply(lambda x: colormap(x))

    # Create a folium map centered on Croatia
    m = folium.Map(location=[45.1, 15.2], zoom_start=7)

    # Function to style each feature based on the color column
    def style_function(feature):
        return {
            'fillColor': feature['properties']['color'],
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.5
        }

    # Filter out rows with missing geometries
    df = df.dropna(subset=['geometry'])

    # Add GeoJSON layer with hover functionality
    folium.GeoJson(
        df,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=['hover_info'], 
            aliases=['Info: '],
            localize=True
        )
    ).add_to(m)

    # Add the colormap to the map
    colormap.add_to(m)

    # Display the map directly in the notebook
    display(m)

    os.makedirs(f"population_census/{folder_name}", exist_ok=True)
    # Update the file name to include the folder name
    file_name = f"population_census/{folder_name}/interactive_map.html"

    m.save(file_name)

In [13]:
def create_static_population_map(df, color_column_name, folder_name):
    # Merge dataframes based on the matching columns
    df = geojson_data.merge(df, on=['Županija', 'Grad/općina'], how='inner')

    # Ensure the necessary columns are present
    if 'Županija' not in df.columns or 'Grad/općina' not in df.columns or 'Ukupno' not in df.columns:
        raise ValueError("The CSV must contain 'Županija', 'Grad/općina', and 'Ukupno' columns.")
    
    # Convert Timestamp columns to strings
    for col in df.select_dtypes(include=['datetime64[ns, UTC]', 'datetime64[ns]']).columns:
        df[col] = df[col].astype(str)
    
    # Check if the color_column_name exists in the DataFrame
    if color_column_name not in df.columns:
        raise ValueError(f"The specified color column '{color_column_name}' does not exist in the CSV file.")
    
    plot = df.plot(
        column=color_column_name, 
        legend=True, 
        figsize=(15, 15)
    )
    plot.set_title(color_column_name, fontsize=14)

    os.makedirs(f"/population_census/{folder_name}", exist_ok=True)
    # Update the file name to include the folder name
    # Replace '/' with '_' in the color_column_name to avoid issues with folder symbols
    color_column_name_safe = color_column_name.replace('/', '')
    file_name = f"/population_census/{folder_name}/{color_column_name_safe}.png"
    # Save the plot as a PNG file
    plot.figure.savefig(file_name, bbox_inches="tight", dpi=300)

    # Filter out rows with missing geometries
    df = df.dropna(subset=['geometry'])

In [14]:
def create_multiple_static_population_maps(df, folder_name):
    # Create the directory if it doesn't exist
    for col in df.columns:
        if col.endswith('%') and col != 'Ukupno, %' and col != 'Ukupno %':
            create_static_population_map(df, col, folder_name)
    

STANOVNIŠTVO PREMA NARODNOSTI PO GRADOVIMA/OPĆINAMA, POPIS 2021.

In [50]:
narodnost = pd.read_csv('data/census/narodnost.csv')

In [ ]:
create_interactive_population_map(narodnost, 'Hrvati, %', 'narodnost')

In [ ]:
create_multiple_static_population_maps(narodnost, 'narodnost')

STANOVNIŠTVO PREMA VJERI PO GRADOVIMA/OPĆINAMA, POPIS 2021.

In [11]:
vjera = pd.read_csv('data/census/vjera.csv')

In [ ]:
create_interactive_population_map(vjera, 'Katolici, %', 'vjera')

In [ ]:
create_multiple_static_population_maps(vjera, 'vjera')

STANOVNIŠTVO PREMA DRŽAVLJANSTVU PO GRADOVIMA/OPĆINAMA, POPIS 2021.

In [14]:
državljanstvo = pd.read_csv('data/census/državljanstvo.csv')

In [ ]:
create_interactive_population_map(državljanstvo, 'Hrvatsko (ukupno), %', 'državljanstvo')

In [ ]:
create_multiple_static_population_maps(državljanstvo, 'državljanstvo')

STANOVNIŠTVO PREMA MATERINSKOM JEZIKU PO GRADOVIMA/OPĆINAMA, POPIS 2021.

In [17]:
materinski_jezik = pd.read_csv('data/census/materinski_jezik.csv')

In [ ]:
create_interactive_population_map(materinski_jezik, 'Hrvatski, %', 'materinski_jezik')

In [ ]:
create_multiple_static_population_maps(materinski_jezik, 'materinski_jezik')

STANOVNIŠTVO PREMA BRAČNOM STATUSU PO GRADOVIMA/OPĆINAMA, POPIS 2021.

In [20]:
bračni_status = pd.read_csv('data/census/bračni_status.csv')
bračni_status = bračni_status[bračni_status['Starost'] == 'Ukupno']

In [ ]:
create_interactive_population_map(bračni_status, 'Neoženjen %', 'bračni_status')

In [ ]:
create_multiple_static_population_maps(bračni_status, 'bračni_status')

KONTINGENTI STANOVNIŠTVA PO GRADOVIMA/OPĆINAMA, POPIS 2021.

In [43]:
dob = pd.read_csv('data/census/dob.csv')
dob = dob[dob['Spol'] == 'sv.']
dob = dob.drop(columns=['Žensko stanovništvo 15 - 49 godina %', 'Žensko stanovništvo 20 - 29 godina %'])


In [ ]:
create_interactive_population_map(dob, 'Radno sposobno stanovništvo %', 'dob')

In [ ]:
create_multiple_static_population_maps(dob, 'dob')

STANOVNIŠTVO STARO 15 I VIŠE GODINA PREMA NAJVIŠOJ ZAVRŠENOJ ŠKOLI, STAROSTI I SPOLU PO GRADOVIMA/OPĆINAMA, POPIS 2021.


In [26]:
završena_škola = pd.read_csv('data/census/završena_škola.csv')
završena_škola = završena_škola[(završena_škola['Starost'] == 'Ukupno') & (završena_škola['Spol'] == 'sv.')]

In [ ]:
create_interactive_population_map(završena_škola, 'Srednja škola %', 'završena_škola')

In [ ]:
create_multiple_static_population_maps(završena_škola, 'završena_škola')

STANOVNIŠTVO PREMA POHAĐANJU ŠKOLE I SPOLU PO GRADOVIMA/OPĆINAMA, POPIS 2021.


In [29]:
pohađanje_škole = pd.read_csv('data/census/pohađanje_škole.csv')
pohađanje_škole = pohađanje_škole[pohađanje_škole['Spol'] == 'sv.']

In [ ]:
create_interactive_population_map(pohađanje_škole, 'Srednja škola (ukupno) %', 'pohađanje_škole')

In [ ]:
create_multiple_static_population_maps(pohađanje_škole, 'pohađanje_škole')

ŽENE STARE 15 I VIŠE GODINA PREMA STAROSTI I BROJU ŽIVOROĐENE DJECE PO GRADOVIMA/OPĆINAMA, POPIS 2021.


In [32]:
broj_živorođene_djece = pd.read_csv('data/census/broj_živorođene_djece.csv')
broj_živorođene_djece = broj_živorođene_djece[broj_živorođene_djece['Starost'] == 'Ukupno']

In [ ]:
create_interactive_population_map(broj_živorođene_djece, 'Rodile jedno ili više djece %', 'broj_živorođene_djece')

In [ ]:
create_multiple_static_population_maps(broj_živorođene_djece, 'broj_živorođene_djece')

STANOVNIŠTVO STARO 15 I VIŠE GODINA PREMA EKONOMSKOJ AKTIVNOSTI, STAROSTI I SPOLU PO GRADOVIMA/OPĆINAMA, POPIS 2021.


In [35]:
ekonomska_aktivnost = pd.read_csv('data/census/ekonomska_aktivnost.csv')
ekonomska_aktivnost = ekonomska_aktivnost[(ekonomska_aktivnost['Starost'] == 'Ukupno') & (ekonomska_aktivnost['Spol'] == 'sv.')]

In [ ]:
create_interactive_population_map(ekonomska_aktivnost, 'Zaposleni %', 'ekonomska_aktivnost')

In [ ]:
create_multiple_static_population_maps(ekonomska_aktivnost, 'ekonomska_aktivnost')

ZAPOSLENI PREMA ZANIMANJU, STAROSTI I SPOLU PO GRADOVIMA/OPĆINAMA, POPIS 2021.


In [21]:
zaposlenost_prema_zanimanju = pd.read_csv('data/census/zaposlenost_prema_zanimanju.csv')
zaposlenost_prema_zanimanju = zaposlenost_prema_zanimanju[(zaposlenost_prema_zanimanju['Starost'] == 'Ukupno') & (zaposlenost_prema_zanimanju['Spol'] == 'sv.')]

In [ ]:
create_interactive_population_map(zaposlenost_prema_zanimanju, 'Poljoprivrednici/poljoprivrednice, šumari/šumarke, ribari/ribarke, lovci/lovkinje %', 'zaposlenost_prema_zanimanju')

In [ ]:
create_multiple_static_population_maps(zaposlenost_prema_zanimanju, 'zaposlenost_prema_zanimanju')

ZAPOSLENI PREMA PODRUČJIMA DJELATNOSTI, STAROSTI I SPOLU PO GRADOVIMA/OPĆINAMA, POPIS 2021.


In [25]:
područje_djelatnosti = pd.read_csv('data/census/područje_djelatnosti.csv')
područje_djelatnosti = područje_djelatnosti[(područje_djelatnosti['Starost'] == 'Ukupno') & (područje_djelatnosti['Spol'] == 'sv.')]


In [ ]:
create_interactive_population_map(područje_djelatnosti, 'A Poljoprivreda, šumarstvo i ribarstvo %', 'područje_djelatnosti')

In [ ]:
create_multiple_static_population_maps(područje_djelatnosti, 'područje_djelatnosti')

ZAPOSLENI PREMA POLOŽAJU U ZAPOSLENJU, STAROSTI I SPOLU PO GRADOVIMA/OPĆINAMA, POPIS 2021.


In [27]:
položaj_u_zaposlenju = pd.read_csv('data/census/položaj_u_zaposlenju.csv')
položaj_u_zaposlenju = položaj_u_zaposlenju[(položaj_u_zaposlenju['Starost'] == 'Ukupno') & (položaj_u_zaposlenju['Spol'] == 'sv.')]

In [ ]:
create_interactive_population_map(položaj_u_zaposlenju, 'Zaposlenici %', 'položaj_u_zaposlenju')

In [ ]:
create_multiple_static_population_maps(položaj_u_zaposlenju, 'položaj_u_zaposlenju')

STANOVNIŠTVO PREMA MIGRACIJSKIM OBILJEŽJIMA I SPOLU PO GRADOVIMA/OPĆINAMA, POPIS 2021.


In [33]:
migracijska_obilježja = pd.read_csv('data/census/migracijska_obilježja.csv')
migracijska_obilježja = migracijska_obilježja.rename(columns={'Ukupan broj stanovnika': 'Ukupno'})
migracijska_obilježja = migracijska_obilježja[(migracijska_obilježja['Spol'] == 'sv.')]

In [ ]:
create_interactive_population_map(migracijska_obilježja, 'Od rođenja stanuju u istom naselju %', 'migracijska_obilježja')

In [ ]:
create_multiple_static_population_maps(migracijska_obilježja, 'migracijska_obilježja')

STANOVNIŠTVO PREMA MJESTU ROĐENJA I SPOLU PO GRADOVIMA/OPĆINAMA, POPIS 2021.


In [35]:
mjesto_rođenja_stanovanja = pd.read_csv('data/census/mjesto_rođenja_stanovanja.csv')
mjesto_rođenja_stanovanja = mjesto_rođenja_stanovanja.rename(columns={'Ukupan broj stanovnika': 'Ukupno'})
mjesto_rođenja_stanovanja = mjesto_rođenja_stanovanja[(mjesto_rođenja_stanovanja['Spol'] == 'sv.')]

In [ ]:
create_interactive_population_map(mjesto_rođenja_stanovanja, 'Rođeni u republici hrvatskoj (ukupno) %', 'mjesto_rođenja_stanovanja')  

In [ ]:
create_multiple_static_population_maps(mjesto_rođenja_stanovanja, 'mjesto_rođenja_stanovanja')